# 1. landast8/9에서 6가지 지표특성 변수 추출 코드 (최종)
- 'LST', 'NDVI', 'NDBI','NDWI', 'Albedo', 'SAVI'
- 위성영상 -> GEE로 추출하여 변수 값 반환

In [1]:
# ============================================
# STEP 0: Earth Engine 인증 (최초 1회만)
# ============================================

import ee

try:
    ee.Initialize()
    print("이미 인증됨")
except:
    print("인증 필요 - 브라우저가 열립니다")
    ee.Authenticate()
    ee.Initialize(project='hongik-graduate-project')
    print("인증 완료!")

이미 인증됨


In [2]:
# ============================================
# STEP 1: 모듈 임포트
# ============================================

import ee
import geopandas as gpd
import pandas as pd
import time
from datetime import datetime

In [ ]:
# ============================================
# STEP 2: 행정동 로드
# ============================================

# SHP 파일 경로
shp_path = r"C:\Users\cub72\Desktop\2025-2\시스템분석\dataset\서울시행정동\BND_ADM_DONG_PG.shp"

all_dong_gdf = gpd.read_file(shp_path, encoding='euc-kr')

# 서울시만
seoul_dong_gdf = all_dong_gdf[all_dong_gdf['ADM_CD'].str.startswith('11')].copy()
seoul_dong_gdf = seoul_dong_gdf.reset_index(drop=True)
seoul_dong_gdf = seoul_dong_gdf.to_crs('EPSG:4326')

# 매핑 테이블
dong_mapping = seoul_dong_gdf[['ADM_CD', 'ADM_NM']].copy()
dong_mapping['idx'] = dong_mapping.index

print(f"행정동 로드 완료: {len(seoul_dong_gdf)}개")

행정동 로드 완료: 426개


In [ ]:
# ============================================
# step3 : Earth Engine FeatureCollection 생성 (대안)
# ============================================

print("FeatureCollection 생성 중...")
start_time = time.time()

# 방법 2: GeoPandas에서 먼저 단순화
seoul_dong_simplified = seoul_dong_gdf.copy()
seoul_dong_simplified['geometry'] = seoul_dong_simplified['geometry'].simplify(0.001)

seoul_features = []
for idx, row in seoul_dong_simplified.iterrows():
    # 이미 단순화된 geometry 사용
    geom = ee.Geometry(row['geometry'].__geo_interface__)
    
    feature = ee.Feature(
        geom,
        {'idx': int(idx)}
    )
    
    seoul_features.append(feature)

# 작은 배치로 나누어 생성
batch_size = 100
seoul_fc = None

for i in range(0, len(seoul_features), batch_size):
    batch = seoul_features[i:i+batch_size]
    batch_fc = ee.FeatureCollection(batch)
    
    if seoul_fc is None:
        seoul_fc = batch_fc
    else:
        seoul_fc = seoul_fc.merge(batch_fc)

elapsed = time.time() - start_time
print(f"FeatureCollection 생성 완료")
print(f"소요 시간: {elapsed:.1f}초")

FeatureCollection 생성 중...
FeatureCollection 생성 완료
   소요 시간: 0.5초


In [5]:
# ============================================
# step4: 데이터 수집 및 지표 계산 함수 정의
# ============================================
# 1. get_landsat8_collection: 날짜/지역별 위성 이미지 수집
# 2. calculate_all_indices: LST, NDVI 등 6개 지표 계산
# 3. get_season: 월 → 계절 변환
# ============================================

def get_landsat8_collection(start_date, end_date, region, cloud_threshold=40):
    """
    Landsat-8과 Landsat-9 데이터를 함께 수집

    Parameters:
        start_date: 시작일 (YYYY-MM-DD)
        end_date: 종료일 (YYYY-MM-DD)
        region: 관심 지역 (FeatureCollection)
        cloud_threshold: 구름 비율 임계값 (%)
    
    Returns:
        ImageCollection    
    """
    # Landsat 8 수집
    l8 = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
          .filterDate(start_date, end_date)
          .filterBounds(region)
          .filter(ee.Filter.lt('CLOUD_COVER', cloud_threshold)))
    
    # Landsat 9 수집
    l9 = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
          .filterDate(start_date, end_date)
          .filterBounds(region)
          .filter(ee.Filter.lt('CLOUD_COVER', cloud_threshold)))
    
    # 두 컬렉션 병합
    return l8.merge(l9)


def calculate_all_indices(image):
    """
    6개 환경 지표 계산
    
    지표:
        LST: 지표면 온도 (°C)
        NDVI: 식생지수 (-1~1)
        NDBI: 건물지수 (-1~1)
        NDWI: 수분지수 (-1~1)
        Albedo: 반사율 (0~1)
        SAVI: 토양보정식생지수 (-1~1)
    """
    # 1. LST (지표면 온도) 계산
    lst = image.select('ST_B10').multiply(0.00341802).add(149.0).subtract(273.15)
    lst = lst.updateMask(lst.gt(-50).And(lst.lt(70))).rename('LST')
    
    # 2. 반사율 변환 (Scale Factor 적용)
    nir = image.select('SR_B5').multiply(0.0000275).add(-0.2)   # 근적외선
    red = image.select('SR_B4').multiply(0.0000275).add(-0.2)   # 적색
    green = image.select('SR_B3').multiply(0.0000275).add(-0.2) # 녹색
    swir = image.select('SR_B6').multiply(0.0000275).add(-0.2)  # 단파적외선
    blue = image.select('SR_B2').multiply(0.0000275).add(-0.2)  # 청색
    
    # 3. NDVI (식생지수) = (NIR - Red) / (NIR + Red)
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    ndvi = ndvi.updateMask(ndvi.gte(-1).And(ndvi.lte(1)))
    
    # 4. NDBI (건물지수) = (SWIR - NIR) / (SWIR + NIR)
    ndbi = swir.subtract(nir).divide(swir.add(nir)).rename('NDBI')
    ndbi = ndbi.updateMask(ndbi.gte(-1).And(ndbi.lte(1)))
    
    # 5. NDWI (수분지수) = (Green - NIR) / (Green + NIR)
    ndwi = green.subtract(nir).divide(green.add(nir)).rename('NDWI')
    ndwi = ndwi.updateMask(ndwi.gte(-1).And(ndwi.lte(1)))
    
    # 6. Albedo (반사율) - 가중평균
    albedo = (blue.multiply(0.356)
              .add(green.multiply(0.130))
              .add(red.multiply(0.373))
              .add(nir.multiply(0.085))
              .add(swir.multiply(0.072))
              .subtract(0.0018)).divide(1.016).rename('Albedo')
    albedo = albedo.updateMask(albedo.gte(0).And(albedo.lte(1)))
    
    # 7. SAVI (토양보정식생지수) - L=0.5
    L = 0.5
    savi = (nir.subtract(red)
            .divide(nir.add(red).add(L))
            .multiply(1 + L)
            .rename('SAVI'))
    savi = savi.updateMask(savi.gte(-1).And(savi.lte(1)))
    
    # 모든 지표를 이미지에 추가
    return image.addBands([lst, ndvi, ndbi, ndwi, albedo, savi]) \
                .select(['LST', 'NDVI', 'NDBI', 'NDWI', 'Albedo', 'SAVI'])


def get_season(month):
    """
    월 → 계절 변환
    
    계절 구분:
        DJF: 겨울 (12, 1, 2월)
        MAM: 봄 (3, 4, 5월)
        JJA: 여름 (6, 7, 8월)
        SON: 가을 (9, 10, 11월)
    """
    if month in [6, 7, 8]:
        return 'JJA'
    elif month in [9, 10, 11]:
        return 'SON'
    elif month in [12, 1, 2]:
        return 'DJF'
    else:
        return 'MAM'


print("함수 정의 완료 (Landsat 수집, 지표 계산, 계절 변환)")

함수 정의 완료 (Landsat 수집, 지표 계산, 계절 변환)


In [6]:
# ============================================
# step5:: 월별 데이터 수집 함수
# ============================================
# 핵심 최적화:
# 1. Geometry 제거 (ee.Feature(None, {...}))
# 2. 청크 분할 (50개씩 나눠서 받기)
# ============================================

def collect_monthly_data(year, month, feature_collection, max_threshold=60):
    """
    특정 연월의 426개 행정동 데이터 수집 (적응형 임계값)
    
    Parameters:
        year: 연도 (예: 2024)
        month: 월 (예: 12)
        feature_collection: 행정동 FeatureCollection
        max_threshold: 최대 구름 임계값 (기본 70)
    
    Returns:
        tuple: (data_list, used_threshold) 또는 (None, None)
    """

    # 날짜 범위 설정
    start_date = f"{year}-{month:02d}-01"
    if month == 12:
        end_date = f"{year+1}-01-01"
    else:
        end_date = f"{year}-{month+1:02d}-01"
    
    # 임계값을 30부터 max_threshold까지 5씩 증가시키면서 시도
    for threshold in range(30, max_threshold + 1, 5):
        # 1. Landsat 이미지 수집
        collection = get_landsat8_collection(start_date, end_date,
                                            feature_collection,
                                            cloud_threshold=threshold)
        img_count = collection.size().getInfo()
        
        # 이미지가 없으면 다음 임계값 시도
        if img_count == 0:
            continue
        
        # 2. 6개 지표 계산 및 월평균
        collection_with_indices = collection.map(calculate_all_indices)
        monthly_mean = collection_with_indices.mean()
        
        # 3. 행정동별 통계 추출 함수 정의
        def add_stats(feature):
            stats = monthly_mean.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=feature.geometry(),
                scale=30,
                maxPixels=1e9,
                tileScale=4,
                bestEffort=False
            )
            
            return ee.Feature(None, {
                'idx': feature.get('idx'),
                'LST': stats.get('LST'),
                'NDVI': stats.get('NDVI'),
                'NDBI': stats.get('NDBI'),
                'NDWI': stats.get('NDWI'),
                'Albedo': stats.get('Albedo'),
                'SAVI': stats.get('SAVI')
            })
        
        # 4. 전체 행정동 계산
        results_fc = feature_collection.map(add_stats)
        
        # 5. 청크 단위로 데이터 받기 (50개씩)
        total_features = results_fc.size().getInfo()
        chunk_size = 50
        all_data = []
        
        # 총 청크 개수 계산 및 출력
        total_chunks = (total_features + chunk_size - 1) // chunk_size
        print(f"  총 {total_features}개 동 -> {total_chunks}개 청크로 분할")
        
        for i in range(0, total_features, chunk_size):
            current_chunk = (i // chunk_size) + 1
            chunk = results_fc.toList(chunk_size, i).getInfo()
            
            # 청크 완료 표시
            start_idx = i
            end_idx = min(i + chunk_size, total_features)
            print(f"  청크 {current_chunk} 완료 ({start_idx}~{end_idx})")
            
            for item in chunk:
                props = item['properties']
                props['year'] = year
                props['month'] = month
                props['season'] = get_season(month)
                props['image_count'] = img_count
                all_data.append(props)
        
        print(f"  완료 ({len(all_data)}개 행, {img_count}초)")

        # 6. 결측치 확인 (LST 기준으로 결측이 없어야 성공)
        missing_count = sum(1 for d in all_data if d['LST'] is None)
        
        if missing_count == 0:  # 결측이 없으면 성공
            print(f"    → 임계값 {threshold} 사용 (이미지 {img_count}개, 결측 0개)")
            return all_data, threshold
        else:
            print(f"    임계값 {threshold}: 결측 {missing_count}개 존재, 다음 임계값 시도")
    
    # 모든 임계값을 시도했지만 실패
    print(f"    → 모든 임계값({max_threshold}까지) 시도했으나 결측 제거 실패")
    return None, None

In [37]:
# ==========================================
# step6: 연도별 전체 데이터 수집 (적응형 임계값 버전)
# ==========================================

# 수집할 연도 지정
target_year = 2021

print("="*70)
print(f"수집 시작: {target_year}년 전체 (1~12월)")
print(f"대상: {len(seoul_dong_gdf)}개 행정동")
print("="*70)

year_data_list = []
error_log = []
threshold_log = {}  # 월별 사용된 임계값 기록
year_start_time = time.time()

# 1~12월 반복
for month in range(1, 13):
    month_start_time = time.time()
    print(f"\n[{target_year}-{month:02d}] 수집 중...")
    
    try:
        month_data, used_threshold = collect_monthly_data(target_year, month, seoul_fc, max_threshold=70)
        
        if month_data is None:
            print(f"  데이터 없음")
            error_log.append({'year': target_year, 'month': month, 'reason': 'no_images'})
            threshold_log[month] = None
        else:
            year_data_list.extend(month_data)
            threshold_log[month] = used_threshold
            month_elapsed = time.time() - month_start_time
            print(f"  완료 ({len(month_data)}개 행, {month_elapsed:.0f}초)")
    
    except Exception as e:
        print(f"  오류: {str(e)[:60]}")
        error_log.append({'year': target_year, 'month': month, 'reason': str(e)[:100]})
        threshold_log[month] = 'ERROR'

# 연도 완료
year_elapsed = time.time() - year_start_time
print("\n" + "="*70)
print(f"{target_year}년 수집 완료!")
print("="*70)

# ★★★ 월별 사용된 임계값 출력 ★★★
print(f"\n {target_year}년 월별 구름 임계값 사용 내역:")
print("-" * 40)
for month in range(1, 13):
    threshold = threshold_log.get(month, 'N/A')
    if threshold is None:
        status = "데이터 없음"
    elif threshold == 'ERROR':
        status = "오류 발생"
    else:
        status = f"임계값 {threshold} 사용"
    print(f"  {month:2d}월: {status}")
print("-" * 40)

if len(year_data_list) > 0:
    # DataFrame 생성
    df = pd.DataFrame(year_data_list)
    df = df.merge(dong_mapping, on='idx', how='left')
    
    column_order = [
        'ADM_CD', 'ADM_NM', 'year', 'month', 'season', 'image_count',
        'LST', 'NDVI', 'NDBI', 'NDWI', 'Albedo', 'SAVI'
    ]
    df = df[column_order]
    df = df.sort_values(['month', 'ADM_CD']).reset_index(drop=True)
    
    # CSV 저장
    save_folder = r"C:\Users\cub72\Desktop\2025-2\시스템분석\dataset\landsat8_추출_데이터"
    output_file = f'{save_folder}/seoul_lst_{target_year}_full_final.csv'
    df.to_csv(output_file, index=False, encoding='utf-8-sig')
    
    # 통계 출력
    print(f"\n파일: {output_file}")
    print(f"행 수: {len(df):,}개")
    print(f"수집 월: {df['month'].nunique()}개월")
    print(f"소요 시간: {year_elapsed:.0f}초 ({year_elapsed/60:.1f}분)")
    
    # 월별 결측 현황
    print(f"\n월별 데이터 수집 현황:")
    for month in range(1, 13):
        month_df = df[df['month'] == month]
        if len(month_df) > 0:
            missing = month_df['LST'].isna().sum()
            print(f"  {month:2d}월: {len(month_df)}개 행, 결측 {missing}개 ({missing/len(month_df)*100:.1f}%)")
    print(f"\n데이터 info 확인")
    print(f"{df.info()}")
else:
    print("수집된 데이터가 없습니다.")

# 오류 로그 출력
if error_log:
    print(f"\n 오류 발생 월: {len(error_log)}개")
    for err in error_log:
        print(f"  {err['year']}-{err['month']:02d}: {err['reason']}")

수집 시작: 2021년 전체 (1~12월)
대상: 426개 행정동

[2021-01] 수집 중...
  총 426개 동 -> 9개 청크로 분할
  청크 1 완료 (0~50)
  청크 2 완료 (50~100)
  청크 3 완료 (100~150)
  청크 4 완료 (150~200)
  청크 5 완료 (200~250)
  청크 6 완료 (250~300)
  청크 7 완료 (300~350)
  청크 8 완료 (350~400)
  청크 9 완료 (400~426)
  완료 (426개 행, 2초)
    → 임계값 30 사용 (이미지 2개, 결측 0개)
  완료 (426개 행, 44초)

[2021-02] 수집 중...
  총 426개 동 -> 9개 청크로 분할
  청크 1 완료 (0~50)
  청크 2 완료 (50~100)
  청크 3 완료 (100~150)
  청크 4 완료 (150~200)
  청크 5 완료 (200~250)
  청크 6 완료 (250~300)
  청크 7 완료 (300~350)
  청크 8 완료 (350~400)
  청크 9 완료 (400~426)
  완료 (426개 행, 3초)
    → 임계값 30 사용 (이미지 3개, 결측 0개)
  완료 (426개 행, 45초)

[2021-03] 수집 중...
  총 426개 동 -> 9개 청크로 분할
  청크 1 완료 (0~50)
  청크 2 완료 (50~100)
  청크 3 완료 (100~150)
  청크 4 완료 (150~200)
  청크 5 완료 (200~250)
  청크 6 완료 (250~300)
  청크 7 완료 (300~350)
  청크 8 완료 (350~400)
  청크 9 완료 (400~426)
  완료 (426개 행, 2초)
    → 임계값 30 사용 (이미지 2개, 결측 0개)
  완료 (426개 행, 44초)

[2021-04] 수집 중...
  총 426개 동 -> 9개 청크로 분할
  청크 1 완료 (0~50)
  청크 2 완료 (50~100)
  청크 3 완료 (100~150)


# 2. 결측 해결

## 2.1 결측있는 연도+달 정리
- 2018-08: 임계값 35에서 결측1개 존재 -> 대치 가능여부 확인 (９０까지도 결측) -> -> **해결가능**
- 2018-06: no_images （９０까지 시도, 결측 ３９９）
- 2019-04: no_images (80까지 시도)
- 2019-07: no_images (80까지 시도)
- 2020-11: no_images (80까지 시도, 결측４０４개)
- 2021-05: no_images (7０까지 시도, 결측 ３７７) -> **해결 가능**

=> 해결 : 
1. 2018-08 : 보간
2. 2018-06, 2019-07 : 여름철이므로, 모델링 결과 왜곡 영향력 때문에 제외
+ 모델 검증 시 : 포함/제외 두 버전 사용도 고려 가능
3. 2020-11,2021-05,2019-04 : 95까지 시도 후 결측 100개 미만이면 사용

In [ ]:
# ==========================================
# 단일 연도-월 지정 임계값으로 바로 수집
# ==========================================

# ★★★ 수집할 연도, 월, 임계값 지정 ★★★
target_year = 2018
target_month = 8
fixed_threshold = 35

print("="*70)
print(f"단일 월 데이터 수집: {target_year}-{target_month:02d} (임계값 {fixed_threshold})")
print("="*70)

start_time = time.time()

# 날짜 범위 설정
start_date = f"{target_year}-{target_month:02d}-01"
if target_month == 12:
    end_date = f"{target_year+1}-01-01"
else:
    end_date = f"{target_year}-{target_month+1:02d}-01"

# 1. Landsat 이미지 수집
collection = get_landsat8_collection(start_date, end_date, seoul_fc, cloud_threshold=fixed_threshold)
img_count = collection.size().getInfo()

print(f"수집된 이미지: {img_count}개")

if img_count == 0:
    print("이미지가 없습니다.")
else:
    # 2. 6개 지표 계산 및 월평균
    collection_with_indices = collection.map(calculate_all_indices)
    monthly_mean = collection_with_indices.mean()
    
    # 3. 행정동별 통계 추출
    def add_stats(feature):
        stats = monthly_mean.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=feature.geometry(),
            scale=30,
            maxPixels=1e9,
            tileScale=4,
            bestEffort=False
        )
        
        return ee.Feature(None, {
            'index': feature.get('idx'),
            'LST': stats.get('LST'),
            'NDVI': stats.get('NDVI'),
            'NDBI': stats.get('NDBI'),
            'NDWI': stats.get('NDWI'),
            'Albedo': stats.get('Albedo'),
            'SAVI': stats.get('SAVI')
        })
    
    results_fc = seoul_fc.map(add_stats)
    
    # 4. 청크 단위로 데이터 받기
    total_features = results_fc.size().getInfo()
    chunk_size = 50
    all_data = []
    
    print(f"총 {total_features}개 동 -> {(total_features + chunk_size - 1) // chunk_size}개 청크로 분할")
    
    for i in range(0, total_features, chunk_size):
        current_chunk = (i // chunk_size) + 1
        chunk = results_fc.toList(chunk_size, i).getInfo()
        
        start_idx = i
        end_idx = min(i + chunk_size, total_features)
        print(f"청크 {current_chunk} 완료 ({start_idx}~{end_idx})")
        
        for item in chunk:
            props = item['properties']
            all_data.append({
                'index': props['index'],
                'year': target_year,
                'month': target_month,
                'season': get_season(target_month),
                'image_count': img_count,
                'LST': props.get('LST'),
                'NDVI': props.get('NDVI'),
                'NDBI': props.get('NDBI'),
                'NDWI': props.get('NDWI'),
                'Albedo': props.get('Albedo'),
                'SAVI': props.get('SAVI')
            })
    
    # 5. DataFrame 생성
    df = pd.DataFrame(all_data)
    df = df.merge(dong_mapping, on='idx', how='left')
    
    column_order = [
        'ADM_CD', 'ADM_NM', 'year', 'month', 'season', 'image_count',
        'LST', 'NDVI', 'NDBI', 'NDWI', 'Albedo', 'SAVI'
    ]
    df = df[column_order]
    df = df.sort_values(['ADM_CD']).reset_index(drop=True)
    
    # 6. CSV 저장
    save_folder = r"C:\Users\cub72\Desktop\2025-2\시스템분석\dataset\landsat8_추출_데이터"
    output_file = f'{save_folder}/seoul_lst_{target_year}{target_month:02d}_threshold{fixed_threshold}.csv'
    df.to_csv(output_file, index=False, encoding='utf-8-sig')
    
    # 7. 결과 출력
    elapsed = time.time() - start_time
    missing = df['LST'].isna().sum()
    
    print("\n" + "="*70)
    print("수집 완료!")
    print("="*70)
    print(f"저장 파일: {output_file}")
    print(f"행 수: {len(df)}개")
    print(f"결측: {missing}개 ({missing/len(df)*100:.1f}%)")
    print(f"소요 시간: {elapsed:.0f}초")

## 2.2 2020-11월 결측 1달 발생 -> 보간하기
- 방법 : Bias-corrected Climatology
- 선형 보간 

In [ ]:
import pandas as pd
from pathlib import Path

base_dir = Path(r"C:\Users\cub72\Desktop\2025-2\시스템분석\dataset\landsat8_추출_데이터")
template_path = base_dir / "seoul_lst_202011.csv"              # 방금 만든 빈 CSV
csv_paths    = sorted(base_dir.glob("seoul_lst_20*_full_final.csv"))

# 1) 11월 데이터 모으기
df_nov = pd.concat([pd.read_csv(p, encoding="utf-8-sig").query("month == 11")
                    for p in csv_paths],
                   ignore_index=True)

# 2) 다년 평균(2020 제외) 계산
num_cols = ["LST", "NDVI", "NDBI", "NDWI", "Albedo", "SAVI"]
clim = (df_nov.query("year != 2020")
              .groupby("ADM_CD")[num_cols]
              .mean())

# 3) 템플릿 읽어와서 값 채우기
tpl = pd.read_csv(template_path, encoding="utf-8-sig")
tpl[num_cols] = tpl["ADM_CD"].map(clim.to_dict("index")).apply(pd.Series).values
tpl["image_count"] = 0              # 필요하면 평균값 등으로 변경
tpl["imputed"] = "climatology"      # 플래그

# # 4) 저장
# out_path = base_dir / "seoul_lst_202011_filled_clim.csv"
# tpl.to_csv(out_path, index=False, encoding="utf-8-sig")

print(f"2020-11 월 climatology 채우기 완료 → {out_path}")

2020-11 월 climatology 채우기 완료 → C:\Users\cub72\Desktop\2025-2\시스템분석\dataset\landsat8_추출_데이터\seoul_lst_202011_filled_clim.csv
